In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('./computer_gpu.csv')

In [3]:
dataset.head()

,Workgrp_m,Workgrp_n,Workgrp_k,Local_m,Local_n,Mem_m,Mem_n,Kernel_unroll,VectorWidth_m,VectorWidth_n,Stride_m,Stride_n,Cache_A,Cache_B,Runtime
0,16,16,16,8,8,8,8,2,1,1,0,0,0,0,116.3700
1,16,16,16,8,8,8,8,2,1,1,0,0,0,1,78.7050
2,16,16,16,8,8,8,8,2,1,1,0,0,1,0,80.5650
3,16,16,16,8,8,8,8,2,1,1,0,0,1,1,86.6375
4,16,16,16,8,8,8,8,2,1,1,0,1,0,0,118.6625


## Computer engineering: GPU workload design

### Dataset: project_datasets/computer_gpu.csv

#### Dataset description: This is a dataset from running an OpenCL benchmark on an AMD GPU. This benchmark breaks up matrix math by resizing a very large matrix (or large number of matrices) into matrices that can fit in hardware memory and cache by using a third dimension. Columns:

###### 1.Workgrp_m (Workgroup size (number of compute units) used for first dimension of matrices)
###### 2.Workgrp_n (Workgroup size (number of compute units) used for second dimension of matrices)
###### 3.Workgrp_k (Workgroup size (number of compute units) used for third dimension of matrices)
###### 4.Local_m (Local workgroup size (number of kernels running on one compute unit) used for first dimension of matrices)
###### 5.Local_n (Local workgroup size (number of kernels running on one compute unit) used for second dimension of matrices)
###### 6.Mem_m (Local memory length used for first dimension of matrices)
###### 7.Mem_n (Local memory length used for second dimension of matrices)
###### 8.Kernel_unroll (Number of times loops are unrolled)
###### 9.VectorWidth_m (Width of vector instruction used for first dimension of matrices)
###### 10.VectorWidth_n (Width of vector instruction used for second dimension of matrices)
###### 11.Stride_m (Use of off-chip memory for the first dimension of matrices)
###### 12.Stride_n (Use of off-chip memory for the second dimension of matrices)
###### 13.Cache_A (Use of caching scheme A)
###### 14.Cache_B (Use of caching scheme B)
###### 15.Runtime (target, runtime in ms)

###### Problem: For this GPU, which features of the OpenCL benchmark seem to affect runtime the largest? Which statistical modeling technique works best to predict the runtime based on this data? If so, which features are the most important for your model?

In [4]:
dataset.describe()

,Workgrp_m,Workgrp_n,Workgrp_k,Local_m,Local_n,Mem_m,Mem_n,Kernel_unroll,VectorWidth_m,VectorWidth_n,Stride_m,Stride_n,Cache_A,Cache_B,Runtime
count,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000,241600.000000
mean,80.415364,80.415364,25.513113,13.935894,13.935894,17.371126,17.371126,5.000000,2.448609,2.448609,0.500000,0.500000,0.500000,0.500000,217.571953
std,42.469220,42.469220,7.855619,7.873662,7.873662,9.389418,9.389418,3.000006,1.953759,1.953759,0.500001,0.500001,0.500001,0.500001,368.750161
min,16.000000,16.000000,16.000000,8.000000,8.000000,8.000000,8.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,13.317500
25%,32.000000,32.000000,16.000000,8.000000,8.000000,8.000000,8.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,40.667500
50%,64.000000,64.000000,32.000000,8.000000,8.000000,16.000000,16.000000,5.000000,2.000000,2.000000,0.500000,0.500000,0.500000,0.500000,69.790000
75%,128.000000,128.000000,32.000000,16.000000,16.000000,32.000000,32.000000,8.000000,4.000000,4.000000,1.000000,1.000000,1.000000,1.000000,228.387500
max,128.000000,128.000000,32.000000,32.000000,32.000000,32.000000,32.000000,8.000000,8.000000,8.000000,1.000000,1.000000,1.000000,1.000000,3341.507500


In [5]:
dataset.head(25)

,Workgrp_m,Workgrp_n,Workgrp_k,Local_m,Local_n,Mem_m,Mem_n,Kernel_unroll,VectorWidth_m,VectorWidth_n,Stride_m,Stride_n,Cache_A,Cache_B,Runtime
0,16,16,16,8,8,8,8,2,1,1,0,0,0,0,116.3700
1,16,16,16,8,8,8,8,2,1,1,0,0,0,1,78.7050
2,16,16,16,8,8,8,8,2,1,1,0,0,1,0,80.5650
3,16,16,16,8,8,8,8,2,1,1,0,0,1,1,86.6375
4,16,16,16,8,8,8,8,2,1,1,0,1,0,0,118.6625
5,16,16,16,8,8,8,8,2,1,1,0,1,0,1,83.1650
6,16,16,16,8,8,8,8,2,1,1,0,1,1,0,84.4350
7,16,16,16,8,8,8,8,2,1,1,0,1,1,1,94.5125
8,16,16,16,8,8,8,8,2,1,1,1,0,0,0,119.0775
9,16,16,16,8,8,8,8,2,1,1,1,0,0,1,86.3025


In [6]:
dataset

,Workgrp_m,Workgrp_n,Workgrp_k,Local_m,Local_n,Mem_m,Mem_n,Kernel_unroll,VectorWidth_m,VectorWidth_n,Stride_m,Stride_n,Cache_A,Cache_B,Runtime
0,16,16,16,8,8,8,8,2,1,1,0,0,0,0,116.3700
1,16,16,16,8,8,8,8,2,1,1,0,0,0,1,78.7050
2,16,16,16,8,8,8,8,2,1,1,0,0,1,0,80.5650
3,16,16,16,8,8,8,8,2,1,1,0,0,1,1,86.6375
4,16,16,16,8,8,8,8,2,1,1,0,1,0,0,118.6625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241595,128,128,32,32,32,32,32,8,4,4,1,0,1,1,17.8175
241596,128,128,32,32,32,32,32,8,4,4,1,1,0,0,36.0350
241597,128,128,32,32,32,32,32,8,4,4,1,1,0,1,35.1600
241598,128,128,32,32,32,32,32,8,4,4,1,1,1,0,28.4525


In [8]:
if dataset.corr()['Workgrp_m']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


In [9]:
if dataset.corr()['Workgrp_n']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


In [10]:
if dataset.corr()['Workgrp_k']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


In [11]:
if dataset.corr()['Local_m']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

Inverse Relation


In [12]:
if dataset.corr()['Local_n']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

Inverse Relation


In [13]:
if dataset.corr()['Mem_m']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

Inverse Relation


In [14]:
if dataset.corr()['Mem_n']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

Inverse Relation


In [15]:
if dataset.corr()['Kernel_unroll']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


In [16]:
if dataset.corr()['VectorWidth_m']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


In [17]:
if dataset.corr()['VectorWidth_n']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


In [18]:
if dataset.corr()['Stride_m']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

Inverse Relation


In [19]:
if dataset.corr()['Stride_n']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

Inverse Relation


In [20]:
if dataset.corr()['Cache_A']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


In [21]:
if dataset.corr()['Cache_B']['Runtime'] < 0:
  print ("Inverse Relation")
else:
  print ("No Pattern")

No Pattern


Local_m , Local_n , Mem_m , Mem_n , Stride_m , Stride_n have effect on Runtime.

In [1]:
from sklearn.linear_model import LinearRegression

In [29]:
dataset.dropna(how='all',inplace=True)

In [67]:
X = np.array([[dataset["Workgrp_m"],dataset['Workgrp_n'],dataset['Workgrp_k'],dataset["Local_m"],dataset["Local_n"],dataset["Mem_m"],dataset["Mem_n"],dataset['Kernel_unroll'],dataset['VectorWidth_m'],dataset['VectorWidth_n'],dataset["Stride_m"]
    ,dataset["Stride_n"],dataset['Cache_A'],dataset['Cache_B']]])

In [68]:
Y = np.array(dataset["Runtime"].values.tolist())

In [69]:
X.shape

(1, 14, 241600)

In [70]:
Y.shape

(241600,)

In [71]:
X = X.reshape(X.shape[1:])

In [72]:
X = X.transpose()

In [73]:
X.shape

(241600, 14)

In [74]:
reg = LinearRegression().fit(X, Y)

In [75]:
reg.score(X, Y)

0.40746998040814775

In [87]:
reg.predict(np.array([16,16,16,8,8,8,8,2,1,1,0,0,0,1]).reshape(1,-1))


array([-63.96933771])